In [1]:
# Imports
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round, col, when, expr
from pydataset import data


# 1. Create a spark data frame that contains your favorite programming languages.
* The name of the column should be language
* View the schema of the dataframe
* Output the shape of the dataframe
* Show the first 5 records in the dataframe

In [2]:
# Create spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# Show range of 5
spark.range(5).show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/19 11:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [3]:
# Create a list of tuples with the values
data = [("python",)] * 20

# Create a PySpark DataFrame
df = spark.createDataFrame(data, ["language"])

# Show the DataFrame
df.show(5)

+--------+
|language|
+--------+
|  python|
|  python|
|  python|
|  python|
|  python|
+--------+
only showing top 5 rows



In [4]:
df.describe().show()

+-------+--------+
|summary|language|
+-------+--------+
|  count|      20|
|   mean|    null|
| stddev|    null|
|    min|  python|
|    max|  python|
+-------+--------+



In [5]:
# Get the number of rows
num_rows = df.count()

# Get the number of columns
num_cols = len(df.columns)

# Display the shape
print(f"Shape: ({num_rows}, {num_cols})")


Shape: (20, 1)


In [6]:
# Print the schema of the DataFrame
df.printSchema()


root
 |-- language: string (nullable = true)



# 2. load the mpg dataset as a spark dataframe.
* Create 1 column of output that contains a message like the one below:
  The 1999 audi a4 has a 4 cylinder engine.
  For each vehicle.
* Transform the trans column so that it only contains either manual or auto.

In [7]:
from pydataset import data
mpg = data('mpg')
mpg = spark.createDataFrame(mpg)
mpg.describe().show()

23/05/19 11:09:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|summary|manufacturer|            model|             displ|             year|              cyl|     trans|drv|               cty|              hwy|  fl|  class|
+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|  count|         234|              234|               234|              234|              234|       234|234|               234|              234| 234|    234|
|   mean|        null|             null| 3.471794871794871|           2003.5|5.888888888888889|      null|4.0|16.858974358974358|23.44017094017094|null|   null|
| stddev|        null|             null|1.2919590310839348|4.509646313320439|1.611534484684289|      null|0.0| 4.255945678889394|5.954643441166446|null|   null|
|    min|        audi|      4runne

In [8]:
# Fill null values with auto(av)
mpg.na.fill('auto(av)', subset="trans").show()


+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [9]:
# Fill null values with the most frequent value
mpg_frequent = mpg.select(col('trans')).groupBy('trans').count().orderBy('count', ascending=False).first()[0]
mpg_filled = mpg.fillna(mpg_frequent, subset=['trans'])
mpg = mpg_filled
mpg_filled.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [10]:
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [11]:
# Select all rows with a4 = 25
result = mpg.filter(mpg.model == 'a4')

# Show the result
result.show()


+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|   a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|   a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+



In [12]:
# Selected a4 models built in 1999 and created a new column with a description
result = mpg.select(col('model'),
                   col('year'),
                   when((col('model') == 'a4') & (col('year') == 1999),
                        'The 1999 Audi A4 has a 4 cylinder engine').otherwise(None)
                   .alias('description'))

# Show the change
result.show()


+------------------+----+--------------------+
|             model|year|         description|
+------------------+----+--------------------+
|                a4|1999|The 1999 Audi A4 ...|
|                a4|1999|The 1999 Audi A4 ...|
|                a4|2008|                null|
|                a4|2008|                null|
|                a4|1999|The 1999 Audi A4 ...|
|                a4|1999|The 1999 Audi A4 ...|
|                a4|2008|                null|
|        a4 quattro|1999|                null|
|        a4 quattro|1999|                null|
|        a4 quattro|2008|                null|
|        a4 quattro|2008|                null|
|        a4 quattro|1999|                null|
|        a4 quattro|1999|                null|
|        a4 quattro|2008|                null|
|        a4 quattro|2008|                null|
|        a6 quattro|1999|                null|
|        a6 quattro|2008|                null|
|        a6 quattro|2008|                null|
|c1500 suburb

In [13]:
mpg.select(
    mpg.model,
    when(mpg.model == 'a4', "'The 1999 Audi A4 has a 4 cylinder engine'")
    .alias("mpg_desc"),
).show(12)



+----------+--------------------+
|     model|            mpg_desc|
+----------+--------------------+
|        a4|'The 1999 Audi A4...|
|        a4|'The 1999 Audi A4...|
|        a4|'The 1999 Audi A4...|
|        a4|'The 1999 Audi A4...|
|        a4|'The 1999 Audi A4...|
|        a4|'The 1999 Audi A4...|
|        a4|'The 1999 Audi A4...|
|a4 quattro|                null|
|a4 quattro|                null|
|a4 quattro|                null|
|a4 quattro|                null|
|a4 quattro|                null|
+----------+--------------------+
only showing top 12 rows



In [14]:
# New column that selects model a4 and year 1999 giving a discription
df = mpg.withColumn('Description', when((col('model') == 'a4') & (col('year') == 1999), 'The 1999 Audi A4 has a 4 cylinder engine'))

# Show the updated change
df.show()



+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|         Description|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|The 1999 Audi A4 ...|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|The 1999 Audi A4 ...|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|                null|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|                null|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|The 1999 Audi A4 ...|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|The 1999 Audi A4 ...|
|        audi|                a4|  3.1|2008|  6|  auto(

# 3. Load the tips dataset as a spark dataframe.
* What percentage of observations are smokers?
* Create a column that contains the tip percentage
* Calculate the average tip percentage for each combination of sex and smoker.


In [15]:
df = data('tips')
df = spark.createDataFrame(df)
df.describe().show()

+-------+------------------+------------------+------+------+----+------+------------------+
|summary|        total_bill|               tip|   sex|smoker| day|  time|              size|
+-------+------------------+------------------+------+------+----+------+------------------+
|  count|               244|               244|   244|   244| 244|   244|               244|
|   mean|19.785942622950813|  2.99827868852459|  null|  null|null|  null| 2.569672131147541|
| stddev| 8.902411954856856|1.3836381890011817|  null|  null|null|  null|0.9510998047322345|
|    min|              3.07|               1.0|Female|    No| Fri|Dinner|                 1|
|    max|             50.81|              10.0|  Male|   Yes|Thur| Lunch|                 6|
+-------+------------------+------------------+------+------+----+------+------------------+



In [16]:
# Get count of all 
total = df.count()

# Get total count of smokers
smokers = df.filter(col('smoker') == 'Yes').count()

# Calculate the percentage of smokers
smokers_percent = (smokers / total) * 100

# Show percentage
print(f"The percentage of smokers in the dataset is: {smokers_percent:.2f}%")


The percentage of smokers in the dataset is: 38.11%


In [17]:
# New column for tip percentage
df = df.withColumn('tip_percentage', round(expr('tip / total_bill * 100'),2))

# Show change
df.show()





+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          5.94|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|         16.05|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|         16.66|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|         13.98|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|         14.68|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|         18.62|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|         22.81|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|         11.61|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|         13.03|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|         21.85|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|         16.65|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|         14.18|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        

In [18]:
# Calculate the average tip percentage for each combination of sex and smoker
avg_tip_percentage = df.groupBy('sex', 'smoker').agg(round(avg('tip_percentage'), 2).alias('avg_tip_percentage'))

# Show
avg_tip_percentage.show()


+------+------+------------------+
|   sex|smoker|avg_tip_percentage|
+------+------+------------------+
|  Male|    No|             16.07|
|Female|    No|             15.69|
|  Male|   Yes|             15.28|
|Female|   Yes|             18.21|
+------+------+------------------+



# 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.
* Convert the temperatures to fahrenheit.
* Which month has the most rain, on average?
* Which year was the windiest?
* What is the most frequent type of weather in January?
* What is the average high and low temperature on sunny days in July in 2013 and 2014?
* What percentage of days were rainy in q3 of 2015?
* For each year, find what percentage of days it rained (had non-zero precipitation).



In [20]:
from pydataset import data
df = data('weather')
df = spark.createDataFrame(df)
df.describe().show()

Did you mean:
water, Wheat, deaths, waders, Weimar, winter, watervoles, bcdeter, veteran, newpainters, Snow.deaths


TypeError: 'NoneType' object is not iterable

23/05/19 17:37:54 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1058300 ms exceeds timeout 120000 ms
23/05/19 17:37:54 WARN SparkContext: Killing executors is not supported by current scheduler.
23/05/19 17:54:44 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.

23/05/19 18:08:49 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/05/19 18:13:06 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

...

23/05/19 21:07:10 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/05/19 21:07:54 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/05/19 21:26:00 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2088)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.base/j

23/05/19 21:42:26 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2088)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.base/j

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 55396)
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/homebrew/anaconda3/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/homebrew/anaconda3/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/homebrew/anaconda3/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt/homebrew/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/opt/homebrew/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/opt/homebrew/anaconda3/lib/python3.9/site-p